# Semantic Triple Suggestion with Sentence-BERT

In [ ]:
!pip install -q sentence-transformers rdflib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 10.7 MB/s eta 0:00:00


In [ ]:
import google.colab
import os
from google.colab import files
uploaded = files.upload()

Saving triples_from_gpt.json to triples_from_gpt.json


In [ ]:
import json

with open('/content/triples_from_gpt.json') as f:
    triples_data = json.load(f)

# Peek into one item
triples_data[0]


{'CQ_ID': 'CQ-E1',
 'CQ_Text': 'What kind of event was [Event]?',
 'Answer': 'Live Aid was a global benefit concert held to raise funds for humanitarian relief.',
 'Triples': [{'subject': 'http://example.org/LiveAid',
   'predicate': 'rdf:type',
   'object': 'schema:MusicEvent'},
  {'subject': 'http://example.org/LiveAid',
   'predicate': 'schema:eventAttendanceMode',
   'object': 'schema:MixedEventAttendanceMode'},
  {'subject': 'http://example.org/LiveAid',
   'predicate': 'schema:audience',
   'object': 'schema:GlobalAudience'},
  {'subject': 'http://example.org/LiveAid',
   'predicate': 'schema:fundedItem',
   'object': 'http://example.org/HumanitarianRelief'},
  {'subject': 'http://example.org/LiveAid',
   'predicate': 'schema:additionalType',
   'object': 'http://example.org/BenefitConcert'}]}

In [ ]:
from sentence_transformers import SentenceTransformer, util
import json
import pandas as pd
import requests

from pprint import pprint


In [ ]:
from rdflib import Graph, RDF, RDFS, OWL
from collections import defaultdict

# Music Meta OWL URL
owl_url = "https://raw.githubusercontent.com/polifonia-project/music-meta-ontology/main/ontology/musicmeta.owl"

# Load RDF graph
g = Graph()
g.parse(owl_url, format='xml')

print("✅ Ontology loaded successfully")


✅ Ontology loaded successfully


In [ ]:
# Namespace compaction (prefix mapping)
namespaces = {
    "mm": "https://w3id.org/polifonia/ontology/music-meta/",
    "mmcore": "https://w3id.org/polifonia/ontology/core/",
    "schema": "http://schema.org/"
}

def compact(uri):
    uri = str(uri)
    for prefix, base in namespaces.items():
        if uri.startswith(base):
            return f"{prefix}:{uri[len(base):]}"
    return uri  # fallback


In [ ]:
# Extract classes
classes = {}
label_to_class = {}
for s in g.subjects(RDF.type, OWL.Class):
    label = compact(s)
    classes[str(s)] = label
    label_to_class[label] = str(s)

# Extract properties (object + datatype)
properties = {}
label_to_property = {}
for ptype in [OWL.ObjectProperty, OWL.DatatypeProperty]:
    for p in g.subjects(RDF.type, ptype):
        prop_uri = str(p)
        label = compact(p)
        properties[prop_uri] = {
            "label": label,
            "domain": set(),
            "range": set()
        }
        label_to_property[label] = prop_uri


In [ ]:
# Domains and Ranges
for p in properties:
    for d in g.objects(p, RDFS.domain):
        properties[p]["domain"].add(str(d))
    for r in g.objects(p, RDFS.range):
        properties[p]["range"].add(str(r))

# Build subject → predicate map
subject_predicate_map = defaultdict(set)
for prop_uri, prop_info in properties.items():
    for domain_uri in prop_info["domain"]:
        subject_predicate_map[domain_uri].add(prop_uri)

# Build predicate → object map
predicate_object_map = defaultdict(set)
for prop_uri, prop_info in properties.items():
    for range_uri in prop_info["range"]:
        predicate_object_map[prop_uri].add(range_uri)


In [ ]:
ontology = {
    "classes": classes,
    "label_to_class": label_to_class,
    "properties": properties,
    "label_to_property": label_to_property,
    "subject_predicate_map": subject_predicate_map,
    "predicate_object_map": predicate_object_map,
    "class_labels": sorted(label_to_class.keys()),
    "property_labels": sorted(label_to_property.keys()),
}


In [ ]:
# Assuming you loaded ontology as in the previous steps
mm_labels = ontology["class_labels"] + ontology["property_labels"]


In [ ]:
from rdflib import Graph, RDFS, RDF, Namespace
import requests

# Define Schema.org namespace
SCHEMA = Namespace("http://schema.org/")

# Download the full schema.org JSON-LD file
schema_url = "https://schema.org/version/latest/schemaorg-current-https.jsonld"
r = requests.get(schema_url)
r.raise_for_status()

# Save locally (optional)
with open("schemaorg.jsonld", "w") as f:
    f.write(r.text)

# Parse using rdflib
g_schema = Graph()
g_schema.parse("schemaorg.jsonld", format="json-ld")



<Graph identifier=N44e613e0a12445a686fb6bc0d0995371 (<class 'rdflib.graph.Graph'>)>

In [ ]:
schema_classes = {}
schema_properties = {}

for s, p, o in g_schema.triples((None, RDF.type, RDFS.Class)):
    label = g_schema.value(subject=s, predicate=RDFS.label)
    if label:
        schema_classes[str(s)] = str(label)

for s, p, o in g_schema.triples((None, RDF.type, RDF.Property)):
    label = g_schema.value(subject=s, predicate=RDFS.label)
    if label:
        schema_properties[str(s)] = str(label)

schema_labels = list(schema_classes.values()) + list(schema_properties.values())
print(f"✅ Extracted {len(schema_labels)} labels from Schema.org.")


✅ Extracted 2427 labels from Schema.org.


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
import pandas as pd

results = []

for item in triples_data:
    cq_id = item.get("CQ_ID", "unknown")
    fact = item["Answer"]

    fact_emb = model.encode(fact, convert_to_tensor=True)
    label_embs = model.encode(schema_labels, convert_to_tensor=True)

    similarities = util.pytorch_cos_sim(fact_emb, label_embs)[0]
    top_k = min(5, len(schema_labels))
    top_results = similarities.topk(k=top_k)

    suggested = [schema_labels[i] for i in top_results.indices]

    results.append({
        "CQ_ID": cq_id,
        "Fact": fact,
        "Top_Suggestions": ", ".join(suggested)
    })

df = pd.DataFrame(results)
df.to_csv("schema_org_suggestions.csv", index=False)
df.head()


,CQ_ID,Fact,Top_Suggestions
0,CQ-E1,Live Aid was a global benefit concert held to ...,"sponsor, funding, FundingAgency, FundingScheme..."
1,CQ-E2,Live Aid was organized to raise funds for fami...,"funding, NGO, FundingScheme, FundingAgency, sp..."
2,CQ-E3,Queen performed at Live Aid held at Wembley St...,"performers, EventReservation, MusicEvent, Stad..."
3,CQ-E3b,David Bowie performed at Live Aid held at Wemb...,"MusicEvent, performers, performer, MusicReleas..."
4,CQ-E5a,London and Philadelphia hosted Live Aid.,"sponsor, Hospital, deathPlace, ApartmentComple..."


In [ ]:
import os
from google.colab import output

# Manually input the API key (safe, not saved in notebook)
GEMINI_API_KEY = input("🔑 Enter your Gemini API key: ")
genai.configure(api_key=GEMINI_API_KEY)


🔑 Enter your Gemini API key: *******


In [ ]:
import time
import random
import google.generativeai as genai
import pandas as pd
import json

# ✅ Load model
model = genai.GenerativeModel(model_name="models/gemini-2.5-flash")

# ✅ Load triples_data
with open("/content/triples_from_gpt.json") as f:
    triples_data = json.load(f)

# ✅ Function to split into atomic facts
def split_into_atomic_facts_with_retry(fact_text, max_retries=5, delay=5):
    prompt = f"""
You are a knowledge engineer. Break the following sentence into atomic factual statements.
Each atomic fact should:
- Be grammatically complete
- Avoid pronouns like "it" or "they"
- Be suitable for RDF-style modeling

Original sentence:
"{fact_text}"

Atomic facts:
1.
"""
    for attempt in range(max_retries):
        try:
            wait = delay * (2 ** attempt) + random.uniform(0, 2)
            print(f"Waiting {wait:.2f}s...")
            time.sleep(wait)
            response = model.generate_content(prompt)
            return [line.strip("0123456789. ").strip() for line in response.text.strip().split("\n") if line.strip()]
        except Exception as e:
            if "429" in str(e):
                wait = delay * (2 ** attempt) + random.uniform(0, 2)
                print(f"Rate limited. Retry {attempt + 1}/{max_retries}. Waiting {wait:.2f}s...")
                time.sleep(wait)
            else:
                print(f"Failed to generate content: {e}")
                return []
    print("Max retries exceeded.")
    return []

# ✅ Process with retry
output_rows = []
for item in triples_data:
    cq_id = item["CQ_ID"]
    answer = item["Answer"]
    atomic_facts = split_into_atomic_facts_with_retry(answer)

    for i, atomic_fact in enumerate(atomic_facts, start=1):
        output_rows.append({
            "CQ_ID": cq_id,
            "Atomic_ID": f"{cq_id}.{i}",
            "Atomic_Fact": atomic_fact
        })

# ✅ Export
df_atomic = pd.DataFrame(output_rows)
df_atomic.to_csv("atomic_facts_output.csv", index=False)
df_atomic.head()


Waiting 5.41s...
Waiting 5.55s...
Waiting 5.99s...
Waiting 6.42s...
Waiting 5.83s...
Waiting 5.65s...
Waiting 5.46s...
Waiting 5.34s...
Waiting 5.96s...
Waiting 6.30s...
Waiting 5.01s...
Waiting 5.67s...
Waiting 6.39s...
Waiting 6.70s...
Waiting 6.85s...
Waiting 6.47s...
Waiting 5.87s...
Waiting 5.78s...
Waiting 5.87s...
Waiting 5.51s...
Waiting 5.50s...
Waiting 6.69s...
Waiting 7.00s...
Waiting 6.67s...
Waiting 6.46s...
Waiting 5.09s...
Waiting 6.38s...
Waiting 5.08s...
Waiting 6.82s...
Waiting 5.91s...
Waiting 5.36s...
Waiting 5.93s...
Waiting 6.94s...
Waiting 5.17s...
Waiting 6.81s...
Waiting 6.80s...
Waiting 6.21s...
Waiting 5.03s...
Waiting 5.59s...
Waiting 6.96s...
Waiting 5.62s...
Waiting 5.19s...
Waiting 5.94s...
Waiting 6.73s...
Waiting 6.99s...
Waiting 5.19s...
Waiting 5.46s...
Waiting 6.02s...
Waiting 5.55s...
Waiting 6.25s...
Waiting 6.16s...
Waiting 6.25s...
Waiting 5.67s...
Waiting 6.00s...
Waiting 6.72s...
Waiting 5.92s...
Waiting 5.65s...
Waiting 6.62s...


,CQ_ID,Atomic_ID,Atomic_Fact
0,CQ-E1,CQ-E1.1,Here are the atomic factual statements:
1,CQ-E1,CQ-E1.2,Live Aid is a concert
2,CQ-E1,CQ-E1.3,Live Aid is a benefit concert
3,CQ-E1,CQ-E1.4,Live Aid has a global scope
4,CQ-E1,CQ-E1.5,Live Aid was held to raise funds


In [ ]:
from google.colab import files
files.download("atomic_facts_output.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load your CSV
df = pd.read_csv("atomic_facts_output.csv")

# Function to identify non-facts
def is_intro_fact(text):
    lowered = text.lower()
    return any(phrase in lowered for phrase in [
        "here are the atomic",
        "atomic facts:",
        "atomic factual statements",
        "derived from the sentence"
    ])

# Filter out intro facts
df_cleaned = df[~df["Atomic_Fact"].apply(is_intro_fact)].copy()

# Reset Atomic_ID within each CQ_ID
def reset_atomic_ids(group):
    group = group.reset_index(drop=True)
    group["Atomic_ID"] = [f"{group['CQ_ID'].iloc[0]}.{i+1}" for i in range(len(group))]
    return group

df_cleaned = df_cleaned.groupby("CQ_ID", group_keys=False).apply(reset_atomic_ids)

# Save to cleaned file
df_cleaned.to_csv("atomic_facts_cleaned.csv", index=False)

# Preview
df_cleaned.head()


/tmp/ipython-input-45-2623323025.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_cleaned = df_cleaned.groupby("CQ_ID", group_keys=False).apply(reset_atomic_ids)


,CQ_ID,Atomic_ID,Atomic_Fact
0,CQ-E1,CQ-E1.1,Live Aid is a concert
1,CQ-E1,CQ-E1.2,Live Aid is a benefit concert
2,CQ-E1,CQ-E1.3,Live Aid has a global scope
3,CQ-E1,CQ-E1.4,Live Aid was held to raise funds
4,CQ-E1,CQ-E1.5,The funds were for humanitarian relief


In [ ]:
from google.colab import files
files.download("atomic_facts_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import time
import google.generativeai as genai

# Load atomic facts (first 100)
df = df_cleaned.head(100)

# Configure Gemini
model = genai.GenerativeModel("models/gemini-2.5-flash")

# Function to generate triple suggestions from atomic fact
def suggest_triples(fact):
    prompt = f"""
You are a knowledge graph engineer.

Given the atomic factual sentence below, suggest 1 to 3 semantic triples that best represent it in RDF-like format.

Use the format:
Subject → Predicate → Object

Sentence:
"{fact}"

Triples:
1.
"""
    try:
        wait = 3 + random.uniform(0, 2)
        print(f"Waiting {wait:.2f}s...")
        time.sleep(wait)
        response = model.generate_content(prompt)
        output = response.text.strip().split("\n")
        triples = [line.strip("1234567890. ").strip() for line in output if "→" in line]
        return triples
    except Exception as e:
        return [f"Error: {e}"]

# Loop through with delay
triple_rows = []

for i, row in df.iterrows():
    cq_id = row["CQ_ID"]
    atomic_id = row["Atomic_ID"]
    atomic_fact = row["Atomic_Fact"]

    print(f"🔄 [{i+1}/100] Processing: {atomic_fact}")
    triples = suggest_triples(atomic_fact)
    triple_rows.append({
        "CQ_ID": cq_id,
        "Atomic_ID": atomic_id,
        "Atomic_Fact": atomic_fact,
        "Suggested_Triples": " | ".join(triples)
    })

    # Wait to avoid rate limiting
    time.sleep(6)  # adjust delay if still getting 429

# Save results
df_triples = pd.DataFrame(triple_rows)
df_triples.to_csv("gemini_triple_suggestions.csv", index=False)
print("✅ Saved to gemini_triple_suggestions.csv")


🔄 [1/100] Processing: Live Aid is a concert
Waiting 3.86s...
🔄 [2/100] Processing: Live Aid is a benefit concert
Waiting 4.95s...
🔄 [3/100] Processing: Live Aid has a global scope
Waiting 4.10s...
🔄 [4/100] Processing: Live Aid was held to raise funds
Waiting 5.00s...
🔄 [5/100] Processing: The funds were for humanitarian relief
Waiting 4.48s...
🔄 [1/100] Processing: Queen performed 'Bohemian Rhapsody' at Live Aid
Waiting 4.79s...
🔄 [2/100] Processing: Queen performed 'Radio Ga Ga' at Live Aid
Waiting 4.36s...
🔄 [3/100] Processing: Queen performed 'We Are the Champions' at Live Aid
Waiting 4.35s...
🔄 [1/100] Processing: Madonna made a debut
Waiting 3.46s...
🔄 [2/100] Processing: The debut occurred in
Waiting 4.77s...
🔄 [3/100] Processing: The debut included a performance
Waiting 4.76s...
🔄 [4/100] Processing: The performance took place in New York City
Waiting 4.03s...
🔄 [1/100] Processing: Queen was formed in
Waiting 4.27s...
🔄 [2/100] Processing: Queen was formed in London
Waiting 3.9

In [ ]:
from google.colab import files
files.download("gemini_triple_suggestions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load pre-trained model
model = SentenceTransformer("all-MiniLM-L6-v2")  # or a larger one if needed

# Load your labels from Music Meta and schema.org
# Assuming `mm_labels` and `schema_labels` are lists of readable labels
mm_label_embeddings = model.encode(mm_labels, convert_to_tensor=True)
schema_label_embeddings = model.encode(schema_labels, convert_to_tensor=True)


In [ ]:
import pandas as pd

df = pd.read_csv("gemini_triple_suggestions.csv")  # Already uploaded
atomic_facts = df['Atomic_Fact'].tolist()
fact_embeddings = model.encode(atomic_facts, convert_to_tensor=True)


In [ ]:
top_k = 5  # number of top suggestions

suggestions = []

for i, emb in enumerate(fact_embeddings):
    mm_sim = util.pytorch_cos_sim(emb, mm_label_embeddings)[0]
    schema_sim = util.pytorch_cos_sim(emb, schema_label_embeddings)[0]

    top_mm = [mm_labels[idx] for idx in mm_sim.topk(top_k).indices]
    top_schema = [schema_labels[idx] for idx in schema_sim.topk(top_k).indices]

    suggestions.append({
        "CQ_ID": df.loc[i, "CQ_ID"],
        "Atomic_Fact": df.loc[i, "Atomic_Fact"],
        "Top_MM_Suggestions": top_mm,
        "Top_Schema_Suggestions": top_schema
    })


In [ ]:
suggestion_df = pd.DataFrame(suggestions)
suggestion_df.to_csv("embedding_based_suggestions.csv", index=False)


In [ ]:
from google.colab import files
files.download("embedding_based_suggestions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import rdflib

# ✅ Load Gemini output CSV
df = pd.read_csv("gemini_triple_suggestions.csv")

# ✅ Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# ✅ Load ontologies
def extract_labels_from_owl(url):
    g = rdflib.Graph()
    g.parse(url)
    label_map = {}
    for s, p, o in g.triples((None, rdflib.RDFS.label, None)):
        if isinstance(o, rdflib.term.Literal):
            label_map[str(s)] = str(o)
    return label_map

mm_labels = extract_labels_from_owl("https://raw.githubusercontent.com/polifonia-project/music-meta-ontology/main/ontology/musicmeta.owl")
schema_labels = extract_labels_from_owl("https://schema.org/version/latest/schemaorg-current-http.rdf")

# ✅ Prepare embeddings
def encode_labels(label_map):
    texts = list(label_map.values())
    embs = model.encode(texts, convert_to_tensor=True)
    return texts, embs

mm_texts, mm_embs = encode_labels(mm_labels)
schema_texts, schema_embs = encode_labels(schema_labels)

# ✅ Suggest top-k matches
def get_top_k_matches(term, label_texts, label_embs, k=3):
    query_emb = model.encode(term, convert_to_tensor=True)
    scores = util.cos_sim(query_emb, label_embs)[0]
    ranked = sorted(zip(label_texts, scores), key=lambda x: x[1], reverse=True)
    return [label for label, score in ranked[:k]]

from urllib.parse import urlparse

# 🌍 Helper: Get QName-style prefix from URI
def format_uri(uri):
    parsed = urlparse(uri)
    if "musicmeta" in parsed.netloc:
        return f"mm:{uri.split('/')[-1]}"
    elif "schema.org" in parsed.netloc:
        return f"schema:{uri.split('/')[-1]}"
    elif "www.w3.org" in parsed.netloc and "rdf-syntax-ns" in parsed.path:
        return f"rdf:{uri.split('#')[-1]}"
    else:
        return uri

# 🔄 Invert label map to get URI from label
def build_inverse_label_map(label_map):
    return {v: k for k, v in label_map.items()}

mm_inverse = build_inverse_label_map(mm_labels)
schema_inverse = build_inverse_label_map(schema_labels)

# 🧠 New function: suggest top-k with formatted URI
def get_top_k_with_uri(term, label_texts, label_embs, inverse_map, k=3):
    query_emb = model.encode(term, convert_to_tensor=True)
    scores = util.cos_sim(query_emb, label_embs)[0]
    ranked = sorted(zip(label_texts, scores), key=lambda x: x[1], reverse=True)

    return [
        format_uri(inverse_map[label])
        for label, score in ranked[:k]
        if label in inverse_map
    ]


# ✅ Parse triples and generate suggestions
rows = []
for _, row in df.iterrows():
    fact = row["Atomic_Fact"]
    suggestions = row["Suggested_Triples"].split("|")

    for triple in suggestions:
        parts = triple.strip().split("→")
        if len(parts) != 3:
            continue
        subj, pred, obj = [p.strip() for p in parts]

        mm_subj = get_top_k_with_uri(subj, mm_texts, mm_embs, mm_inverse)
        mm_pred = get_top_k_with_uri(pred, mm_texts, mm_embs, mm_inverse)
        mm_obj  = get_top_k_with_uri(obj, mm_texts, mm_embs, mm_inverse)

        schema_subj = get_top_k_with_uri(subj, schema_texts, schema_embs, schema_inverse)
        schema_pred = get_top_k_with_uri(pred, schema_texts, schema_embs, schema_inverse)
        schema_obj  = get_top_k_with_uri(obj, schema_texts, schema_embs, schema_inverse)

        rows.append({
            "CQ_ID": row["CQ_ID"],
            "Atomic_ID": row["Atomic_ID"],
            "Atomic_Fact": fact,
            "Subject": subj,
            "Predicate": pred,
            "Object": obj,
            "Top_MM_Subject": mm_subj,
            "Top_MM_Predicate": mm_pred,
            "Top_MM_Object": mm_obj,
            "Top_Schema_Subject": schema_subj,
            "Top_Schema_Predicate": schema_pred,
            "Top_Schema_Object": schema_obj,
        })

# ✅ Save the output
df_all = pd.DataFrame(rows)
df_all.to_csv("semantic_triple_mapping_all_parts.csv", index=False)
df_all.head()


,CQ_ID,Atomic_ID,Atomic_Fact,Subject,Predicate,Object,Top_MM_Subject,Top_MM_Predicate,Top_MM_Object,Top_Schema_Subject,Top_Schema_Predicate,Top_Schema_Object
0,CQ-E1,CQ-E1.1,Live Aid is a concert,Live Aid,rdf:type,Concert,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,"[schema:LivingWithHealthAspect, schema:LiveAlb...","[schema:DataType, schema:mapType, schema:conte...","[schema:performers, schema:stage, schema:Festi..."
1,CQ-E1,CQ-E1.2,Live Aid is a benefit concert,Live Aid,rdf:type,BenefitConcert,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,"[schema:LivingWithHealthAspect, schema:LiveAlb...","[schema:DataType, schema:mapType, schema:conte...","[schema:BenefitsHealthAspect, schema:benefitsS..."
2,CQ-E1,CQ-E1.2,Live Aid is a benefit concert,Live Aid,instanceOf,BenefitConcert,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,"[schema:LivingWithHealthAspect, schema:LiveAlb...","[schema:memberOf, schema:exampleOfWork, schema...","[schema:BenefitsHealthAspect, schema:benefitsS..."
3,CQ-E1,CQ-E1.2,Live Aid is a benefit concert,Live Aid,hasPurpose,CharitableCause,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,"[schema:LivingWithHealthAspect, schema:LiveAlb...","[schema:hasCertification, schema:hasBioChemEnt...","[schema:CharitableIncorporatedOrganization, sc..."
4,CQ-E1,CQ-E1.3,Live Aid has a global scope,Live Aid,hasScope,"""global""",[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,[https://w3id.org/polifonia/ontology/music-met...,"[schema:LivingWithHealthAspect, schema:LiveAlb...","[schema:warrantyScope, schema:WarrantyScope, s...","[schema:InternationalTrial, schema:globalLocat..."


In [ ]:
from google.colab import files
files.download("semantic_triple_mapping_all_parts.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>